In [1]:
import re
import email
from email.parser import Parser

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
emails_df = pd.read_csv('emails.csv')
emails_df.head(1)

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...


In [3]:
def get_parsed_email_body(data, email_sender, email_body):
    email = Parser().parsestr(data)
    if email['from']:
        sender = email['from']
        sender = sender.replace("\n", "")
        sender = sender.replace("\t", "")
        sender = sender.replace(" ", "")

        res = re.search(r'X-FileName.*\n*((?:\n.*)*?)(?=\n.*Forwarded|\n.*Original|\n\n\n|\Z)', data)
        if not res:
            return False
        if not res.group(1):
            return False
        email_body.append(res.group(1))
        email_sender.append(sender)
    return True

def data_cleaning(emails_df):
    filtered_rows = []
    email_sender = []
    email_body = []
    for index, row in emails_df.iterrows():
        if not get_parsed_email_body(row["message"], email_sender, email_body):
            filtered_rows.append(index)
    emails_df.drop("message", axis=1, inplace=True)
    emails_df.drop(filtered_rows, axis=0, inplace=True)
    emails_df["email_body"] = email_body
    emails_df["sender_email"] = email_sender
    emails_df["sender"] = emails_df['file'].map(lambda x:x.split('/')[0])
    

data_cleaning(emails_df)
emails_df.head(1)

,file,email_body,sender_email,sender
0,allen-p/_sent_mail/1.,\nHere is our forecast\n\n,phillip.allen@enron.com,allen-p


In [4]:
valid_dirs = ['sent_items', '_sent_mail', 'sent', '_sent']
emails_df['valid'] = emails_df['file'].map(lambda x : True if x.split('/')[1] in valid_dirs else False)

In [5]:
emails_df.drop(emails_df.index[emails_df['valid'] == False], inplace=True)

In [6]:
sent_mail_check = set(emails_df['file'].map(lambda x:x.split('/')[1]))
print(sent_mail_check)

{'_sent_mail', '_sent', 'sent_items', 'sent'}


In [7]:
emails_df.to_csv('kaggle_enron_email_cleaned.csv')